### 과제 url

url => https://www.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=comment_cnt&page=1

과제: 무신사 코디맵 페이지(많은 댓글 수 기준 정렬)에 나와있는 60개의 코디맵에 대하여 각각 ['codimap_category', 'codimap_title', 'codimap_date', 'views', 'comment_numbers'] 정보를 가져와서 csv 형태로 만든 다음, 해당 코드(.ipynb)와 csv 파일을 본인 github 과제 repository에 제출

Challenge:
1. 각 코디맵마다 클릭한 뒤 다시 이전페이지로 돌아오는 코드를 반복문에 추가하여 ['codimap_explain', 'codimap_hashtag', 'codimap_imgurl'] 정보도 가져오기
2.  page url을 format 함수를 이용해 수정한 뒤 반복문을 통해 1페이지부터 5페이지까지의 코디맵 정보 가져오기

 Challenge는 Optional Task로 필수적으로 해야하는 것은 아닙니다. 좀 더 심화된 크롤링을 원할경우 시도해보면 좋을 것 같아요!

In [1]:
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
print(selenium.__version__)

4.15.2


In [70]:
# ['category', 'title', 'codimap_date', 'views', 'comment_numbers']
# page 1~5
# ['codimap_explain', 'codimap_hashtag', 'codimap_imgurl']

chrome_options = Options()
driver = webdriver.Chrome(options=chrome_options)

codi_list = []
url = 'https://www.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=comment_cnt&page=1'
driver.get(url)
driver.implicitly_wait(2)
try:
    driver.maximize_window()
except:
    pass

page_buttons = ['/html/body/div[3]/div[2]/form/div[4]/div/div[4]/div/div/a[4]',
           '/html/body/div[3]/div[2]/form/div[4]/div/div[4]/div/div/a[5]',
           '/html/body/div[3]/div[2]/form/div[4]/div/div[4]/div/div/a[6]',
           '/html/body/div[3]/div[2]/form/div[4]/div/div[4]/div/div/a[7]']
codi_buttons = ['/html/body/div[3]/div[2]/form/div[4]/div/ul/li[{}]'.format(i) for i in range(1, 61)]


for page in range(5):

    interval = 1
    prev_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

        # Wait to load page
        time.sleep(interval)

        # Calculate new scroll height and compare with last scroll height
        curr_height = driver.execute_script("return document.body.scrollHeight")
        if curr_height == prev_height:
            break
        
        prev_height = curr_height
    

    columns = ['codimap_category', 'codimap_title', 'codimap_date', 'views', 'comment_numbers','codimap_explain', 'codimap_hashtag', 'codimap_imgurl']
    values = []
    soup = BeautifulSoup(driver.page_source, 'lxml')
    data_rows = soup.find_all('li', attrs={'class':'style-list-item'})

    for i, row in enumerate(data_rows):

        print('{0}페이지 {1}번째 데이터를 수집합니다'.format(page+1, i+1))
        blank = []

        # codimap_category
        category = row.find('span', attrs={'class':'style-list-information__text'})
        if category:
            category = category.get_text().strip()
            blank.append(category)
        else:
            blank.append('Something is wrong')
            print('{0}페이지 {1}번째 카테고리 가져올 때 문제발생'.format(page+1, i+1))
            continue
        
        # codimap_title
        title = row.find('strong', attrs={'class':'style-list-information__title'})
        if title:
            title = title.get_text().strip()
            blank.append(title)
        else:
            blank.append('Something is wrong')
            print('{0}페이지 {1}번째 코디 이름 가져올 때 문제발생'.format(page+1, i+1))
            continue
        
        # codimap_date
        date = row.find_all('span', attrs={'class':'post-information__text'})[0]
        if date:
            date = date.get_text().strip()
            blank.append(date)
        else:
            blank.append('Something is wrong')
            print('{0}페이지 {1}번째 날짜 가져올 때 문제발생'.format(page+1, i+1))
            continue
        
        # views
        views = row.find_all('span', attrs={'class':'post-information__text'})[1]
        if views:
            views = views.get_text().strip()
            blank.append(views)
        else:
            blank.append('Something is wrong')
            print('{0}페이지 {1}번째 조회수 가져올 때 문제발생'.format(page+1, i+1))
            continue
        
        # comment_numbers
        comment = row.find_all('span', attrs={'class':'post-information__text'})[2]
        if comment:
            comment = comment.get_text().strip()
            blank.append(comment)
        else:
            blank.append('Something is wrong')
            print('{0}페이지 {1}번째 댓글수 가져올 때 문제발생'.format(page+1, i+1))
            continue
        
        # 코디 상세 페이지로 이동
        codi_button = driver.find_element(By.XPATH, codi_buttons[i-1])
        codi_button.click()
        soup_2 = BeautifulSoup(driver.page_source, 'lxml')
        # style_tag = soup_2.find('div', attrs={'class':'styling-tag'})
        
        # codimap_explain
        explain = soup_2.find('p', attrs={'class':'styling_txt'})
        if explain:
            explain = explain.get_text().strip()
            blank.append(explain)
        else:
            blank.append('Something is wrong')
            print('{0}페이지 {1}번째 코디 설명 가져올 때 문제발생'.format(page+1, i+1))
            continue
        
        # codimap_hashtag
        hashtag = ''
        hashtag_list = soup_2.find_all('a', attrs={'class':'ui-tag-list__item'})
        if hashtag_list:
            for tag in hashtag_list:
                hashtag += tag.get_text().strip() + ' '
            blank.append(hashtag)
        else:
            blank.append('Something is wrong')
            print('{0}페이지 {1}번째 해시태그 가져올 때 문제발생'.format(page+1, i+1))

        
        # codimap_imgurl
        imgurl_row = soup_2.find('div', attrs={'class':'codimap-cont'})
        imgurl = imgurl_row.find('img', attrs={'class':'photo'})
        if imgurl:
            imgurl = imgurl.get('src')
            blank.append(imgurl)
        else:
            blank.append('Something is wrong')
            print('{0}페이지 {1}번째 이미지 가져올 때 문제발생'.format(page+1, i+1))
            continue
        
        driver.back()
        
        
        values.append(blank)
        print('------------------------------------')
        
    page_button = driver.find_element(By.XPATH, page_buttons[page-1])
    page_button.click()
    time.sleep(1)

    df = pd.DataFrame(values, columns=columns)
    codi_list.append(df)    



1페이지 1번째 데이터를 수집합니다
------------------------------------
1페이지 2번째 데이터를 수집합니다
------------------------------------
1페이지 3번째 데이터를 수집합니다
------------------------------------
1페이지 4번째 데이터를 수집합니다
------------------------------------
1페이지 5번째 데이터를 수집합니다
------------------------------------
1페이지 6번째 데이터를 수집합니다
------------------------------------
1페이지 7번째 데이터를 수집합니다
------------------------------------
1페이지 8번째 데이터를 수집합니다
------------------------------------
1페이지 9번째 데이터를 수집합니다
------------------------------------
1페이지 10번째 데이터를 수집합니다
------------------------------------
1페이지 11번째 데이터를 수집합니다
------------------------------------
1페이지 12번째 데이터를 수집합니다
------------------------------------
1페이지 13번째 데이터를 수집합니다
------------------------------------
1페이지 14번째 데이터를 수집합니다
------------------------------------
1페이지 15번째 데이터를 수집합니다
------------------------------------
1페이지 16번째 데이터를 수집합니다
------------------------------------
1페이지 17번째 데이터를 수집합니다
------------------------------------
1페이지 18번째 데이터를 수집합니다
--

In [71]:
df_ = pd.concat(codi_list).reset_index(drop=True)
print(df_.shape)
df_.head(10)

(300, 8)


,codimap_category,codimap_title,codimap_date,views,comment_numbers,codimap_explain,codimap_hashtag,codimap_imgurl
0,캐주얼,톤온톤 코디,21.07.02,"조회 17,328",댓글 50,편하게 입기 좋은 스웨트셔츠와 데님 팬츠를 연출하고 백팩으로 완성한 캐주얼 룩,#오버핏 #워싱 #캠퍼스 #꾸안꾸 #레이어드룩 #남친룩 #가을 #캐주얼 #제이반스 ...,//image.msscdn.net/images/codimap/detail/6495/...
1,캐주얼,블랙 매니아,21.02.23,"조회 36,364",댓글 47,편하게 입기 좋은 반팔 티셔츠와 와이드 팬츠를 연출하고 크로스 백으로 마무리한 캐주얼 룩,#톤온톤 #뉴트럴톤 #캠퍼스 #피크닉 #와이드핏 #오버핏 #꾸안꾸 #여름 #캐주얼 ...,//image.msscdn.net/images/codimap/detail/4035/...
2,캐주얼,오늘의 코디,22.03.22,"조회 13,434",댓글 44,편안한 무드의 조거 팬츠에 블랙 스웨트셔츠를 매치한 후 볼캡으로 마무리한 캐주얼 룩,#봄 #캐주얼 #라퍼지스토어 #다이아몬드 레이라 #나이키 #엠프렌즈 #모티패스트 #...,//image.msscdn.net/images/codimap/detail/1969/...
3,댄디,추천 댄디 룩,21.09.15,"조회 21,629",댓글 32,로고 디테일이 돋보이는 스웨트셔츠와 데님 팬츠를 코디하고 스니커즈로 완성한 캐주얼 룩,#로고플레이 #오버핏 #스티치 #캠퍼스 #봄 #캐주얼 #유니폼브릿지 #아디다스 #4...,//image.msscdn.net/images/codimap/detail/13814...
4,캐주얼,릴랙스 룩,21.02.23,"조회 24,058",댓글 25,담백한 디테일의 니트와 셔츠를 레이어드하고 데님 팬츠를 더해 완성한 댄디 룩,#데이트 #그래픽 #레이어드룩 #가을 #댄디 #엘무드 #뮤지엄바이비컨 #테이크이지 ...,//image.msscdn.net/images/codimap/detail/6746/...
5,스트릿,힙하다 힙해,21.02.03,"조회 36,113",댓글 24,플라워 자수가 눈에 띄는 니트에 조거 팬츠를 더한 후 레드 볼캡으로 컬러 포인트를 ...,#봄 #캐주얼 #카시오 #코닥 #제로 #와릿이즌 #뉴발란스 #커버낫 #디지털 #미니...,//image.msscdn.net/images/codimap/detail/1974/...
6,댄디,데이트 코디,21.12.24,"조회 11,002",댓글 19,레더 블루종 점퍼에 오버사이즈 니트와 조거 팬츠를 매치하고 블루 컬러 볼캡으로 멋을...,#봄 #스트릿 #제로 #월간 #라퍼지스토어 #키치오브제 #블론드나인 #47브랜드 #...,//image.msscdn.net/images/codimap/detail/1701/...
7,캐주얼,심플한 코디,21.07.30,"조회 41,380",댓글 18,깔끔한 무드의 니트에 데님 팬츠를 스타일링하고 스니커즈로 마무리한 댄디 룩,#남친룩 #레더 #데이트 #테이퍼드핏 #오버핏 #소개팅 #스티치 #겨울 #댄디 #쿠...,//image.msscdn.net/images/codimap/detail/10584...
8,댄디,훈훈 남친 룩,21.03.24,"조회 20,029",댓글 17,로고 프린팅이 돋보이는 반팔 티셔츠와 스웨트 팬츠를 매치하고 스니커즈로 완성한 이지...,#로고플레이 #편안한 #원마일웨어 #와이드핏 #여름 #캐주얼 #리 #세미콜론 아이웨...,//image.msscdn.net/images/codimap/detail/5053/...
9,댄디,서머 데이트 룩,21.07.26,"조회 8,378",댓글 17,체크 패턴이 돋보이는 블레이저에 셔츠와 데님 팬츠를 더하고 담백한 구두로 마무리한 ...,#봄 #댄디 #모디파이드 #반 #락포트 #수아레 #슈트/블레이저 재킷 #기타 시계 ...,//image.msscdn.net/images/codimap/detail/2392/...


In [72]:
# csv 파일로 저장
df_.to_csv('codi.csv', encoding = 'utf-8-sig')